<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## *Fabio Peña* 
## *__FpS__*

## Twitter


Hoy mi seleccion Colombia debutó contra Uruguay, pero perdimos, asi que vamos a ver que tuist se encuentran. 

In [1]:
import pandas as pd
import tweepy

In [2]:
# Leer llaves
keys = pd.read_csv("key_demovf_profe.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [3]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [76]:
tweets = tweepy.Cursor(api.search,
                       q=['perdimos'],
                       lang='es',
                       geocode=f'4.649178,-74.062827,4mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [77]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [78]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:34:05,Bogotá,"Berlin, Deutschland",1327424534031589376,https://twitter.com/i/web/status/1327424534031...
1,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:30:18,Bogotá,,1327423578128736256,https://twitter.com/i/web/status/1327423578128...
2,Perdimos partido primordial 😔... #ConPDePolombia,2020-11-14 01:23:16,Bogotá,Bogotá,1327421812154781696,https://twitter.com/i/web/status/1327421812154...
3,RT @julianchoherbal: @PinoCalad Está fue la es...,2020-11-14 01:19:29,Bogotá,Colombia - Santander,1327420857547894785,https://twitter.com/i/web/status/1327420857547...
4,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,2020-11-14 01:09:24,Bogotá,"Bogotá, Colombia",1327418320375656448,https://twitter.com/i/web/status/1327418320375...


In [79]:
ciudades = pd.read_csv(r'C:\Users\ITHELATH\Documents\NLPTW\co_small.csv')
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


Pre-procesamiento para Twitter

In [80]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:34:05,Bogotá
1,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:30:18,Bogotá
2,Perdimos partido primordial 😔... #ConPDePolombia,2020-11-14 01:23:16,Bogotá
3,RT @julianchoherbal: @PinoCalad Está fue la es...,2020-11-14 01:19:29,Bogotá
4,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,2020-11-14 01:09:24,Bogotá


In [81]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [82]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:34:05,Bogotá,RT Perdimos partido primordial 😔...,[],[#ConPDePolombia],[@DanielSamperO:]
1,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:30:18,Bogotá,RT Perdimos partido primordial 😔...,[],[#ConPDePolombia],[@DanielSamperO:]
2,Perdimos partido primordial 😔... #ConPDePolombia,2020-11-14 01:23:16,Bogotá,Perdimos partido primordial 😔...,[],[#ConPDePolombia],[]
3,RT @julianchoherbal: @PinoCalad Está fue la es...,2020-11-14 01:19:29,Bogotá,RT Está fue la estrategia usada por Queiroz ...,[],[],"[@julianchoherbal:, @PinoCalad]"
4,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,2020-11-14 01:09:24,Bogotá,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,[https://t.co/uh8ULWHR1b],"[#animals, #infantil, #naturaleza, #Pequeño, #...",[]


In [83]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [84]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:34:05,Bogotá,RT Perdimos partido primordial 😔...,[],[#ConPDePolombia],[@DanielSamperO:],[😔]
1,RT @DanielSamperO: Perdimos partido primordial...,2020-11-14 01:30:18,Bogotá,RT Perdimos partido primordial 😔...,[],[#ConPDePolombia],[@DanielSamperO:],[😔]
2,Perdimos partido primordial 😔... #ConPDePolombia,2020-11-14 01:23:16,Bogotá,Perdimos partido primordial 😔...,[],[#ConPDePolombia],[],[😔]
3,RT @julianchoherbal: @PinoCalad Está fue la es...,2020-11-14 01:19:29,Bogotá,RT Está fue la estrategia usada por Queiroz ...,[],[],"[@julianchoherbal:, @PinoCalad]",[]
4,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,2020-11-14 01:09:24,Bogotá,ESPECIAL: Aprende a vivir - Edición&gt; Animal...,[https://t.co/uh8ULWHR1b],"[#animals, #infantil, #naturaleza, #Pequeño, #...",[],"[😍, 🤍, 🥰]"


# Visualizaciones usando Plotly


In [85]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
3,2020-11-05,20
8,2020-11-06,5
4,2020-11-07,19
2,2020-11-08,23
5,2020-11-09,10
6,2020-11-10,10
1,2020-11-11,33
9,2020-11-12,4
0,2020-11-13,96
7,2020-11-14,7


In [86]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [87]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#f21d1d'})



layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [88]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
8,😭,14
23,💙,5
17,😂,5
0,😔,4
27,😡,3


In [91]:
trace = go.Bar(x=temp_emojis.emoji.values[:20],
               y=temp_emojis.cnt.values[:20])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)